# Data exploration and processing

Here, we add population data to the original data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install geopandas rtree &> /dev/null # install necessary packages 

import geopandas as gpd
import pandas as pd
import numpy as np
import rtree
import os
import pickle
import matplotlib.pyplot as plt
import sys

os.chdir('/content/drive/MyDrive/shared/ssc22-case-comp/dataset/')
sys.path.append(os.path.abspath("/content/drive/MyDrive/shared/ssc22-case-comp/sonny_dir/custom_modules"))

import weighted_average as WA

print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/shared/ssc22-case-comp/dataset


In [41]:
del WA
import weighted_average as WA

In [33]:
del tqdm
from tqdm.auto import tqdm

In [5]:
file_path = './Nunavut/Nunavut-speed-tiles.shp'
test_data = gpd.read_file(file_path)

test_data['centroid'] = test_data.loc[:, 'geometry'].centroid
test_dest_df = get_dest_df(test_data, dest_col = 'PCUID')

test_data['distance'] = 0

from tqdm import tqdm

for i in tqdm(range(0, test_data.shape[0])): # takes about 20 mins
    temp_row = test_data.loc[i, 'centroid']
    # test_data.loc[i, 'distance'] = calculate_distance(temp, dest_df = test_dest_df)
    test_data.loc[i, 'distance'] = np.min(test_dest_df.distance(temp_row)) / 1000
test_data.head(5)

w_dist = WA.gen_w_avg(test_data, group_col='CDUID', to_avg=['avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'SACTYPE', 'distance'], to_sum=['DA_POP'], weight='tests')
w_dist

100%|██████████| 163/163 [00:00<00:00, 190.39it/s]


,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,year,quarter,conn_type,PRUID,...,DAUID,SACTYPE,DA_POP,PCUID,PCNAME,PCTYPE,PCCLASS,geometry,centroid,distance
0,0300030011013300,14352,3896,62,2,1,2019,Q1,mobile,62,...,62050015,8,895.0,None,None,None,None,"POLYGON ((6609373.822 3108954.900, 6609630.679...",POINT (6609519.911 3108844.311),453.970065
1,0211133010202021,29242,2291,68,1,1,2019,Q2,mobile,62,...,62050020,8,1520.0,1320,Rankin Inlet,4,2,"POLYGON ((6188267.501 2937262.133, 6188539.188...",POINT (6188402.845 2937125.972),0.000000
2,0300131121202333,39944,45873,56,2,1,2019,Q3,fixed,62,...,62040060,8,2320.0,0306,Iqaluit,4,2,"POLYGON ((7292821.313 3241284.005, 7293066.740...",POINT (7292991.161 3241208.640),0.000000
3,0103001132303030,117554,5729,63,1,1,2019,Q4,fixed,62,...,62040059,8,60.0,None,None,None,None,"POLYGON ((6634139.673 5215852.983, 6634214.575...",POINT (6634195.871 5215834.282),2080.085766
4,0103001132321021,15708,11773,52,1,1,2019,Q4,fixed,62,...,62040059,8,60.0,None,None,None,None,"POLYGON ((6634364.732 5215216.106, 6634439.689...",POINT (6634420.967 5215197.388),2079.410375


In [2]:
# apply parallel computing using joblib

from joblib import Parallel, delayed

with open('./python_canada-speed-data.p', 'rb') as file:
    can_data = pickle.load(file)

In [8]:
can_dat = can_data.copy()
prs = can_dat['PRUID'].unique()

In [42]:
def pr_job(pr, distance=False):

    df = can_dat.loc[can_dat['PRUID'] == pr, :].copy()
    
    if distance==True:
        # df = df.loc[:, ['PRUID', 'CDUID', 'tests', 'geometry', 'year', 'quarter','conn_type', 'PCUID', 'PRNAME','DA_POP']]
        df['centroid'] = df.loc[:, 'geometry'].centroid
        df['distance'] = 0
        
        dest_df = WA.get_dest_df(df, dest_col = 'PCUID')

        for i in tqdm(range(0, df.shape[0])): # takes about 20 mins
            temp_row = df['centroid'].iloc[i]
            df.loc[i, 'distance'] = np.min(dest_df.distance(temp_row)) / 1000
        
        result = WA.gen_w_avg(df, group_col='CDUID', to_avg = ['distance'], to_sum = ['DA_POP'], weight='tests')
        return result
    
    else:
        result = WA.gen_w_avg(df, group_col='CDUID', to_avg = ['avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'SACTYPE'], to_sum = ['DA_POP'], weight='tests')
        return result

In [ ]:
from tqdm import tqdm
result_df = pd.concat([pr_job(pr) for pr in tqdm(prs)])

# save outputs
with open('./can-w-avg-cd.p', 'wb') as file:
    pickle.dump(result_df, file)
result_df.to_csv('./can-w-avg-cd.csv', index=False)

In [53]:
# print(prs)
prnames = ['Northwest Territories', 'Yukon', 'British Columbia', 'Alberta', 'Saskatchewan', 'Manitoba', 'Ontario', 'Newfoundland and Labrador', 'Quebec', 'New Brunswick', 'Nova Scotia', 'Prince Edward Island', 'Nunavut']
prr = dict(zip(prs, prnames))
prr

['61' '60' '59' '48' '47' '46' '35' '10' '24' '13' '12' '11' '62']


{'10': 'Newfoundland and Labrador',
 '11': 'Prince Edward Island',
 '12': 'Nova Scotia',
 '13': 'New Brunswick',
 '24': 'Quebec',
 '35': 'Ontario',
 '46': 'Manitoba',
 '47': 'Saskatchewan',
 '48': 'Alberta',
 '59': 'British Columbia',
 '60': 'Yukon',
 '61': 'Northwest Territories',
 '62': 'Nunavut'}

In [90]:
can_dat = can_data.copy()
prs = can_dat['PRUID'].unique()
prnames = ['Northwest Territories', 'Yukon', 'British Columbia', 'Alberta', 'Saskatchewan', 'Manitoba', 'Ontario', 'Newfoundland and Labrador', 'Quebec', 'New Brunswick', 'Nova Scotia', 'Prince Edward Island', 'Nunavut']
prr = dict(zip(prs, prnames))

def pr_dist(can_dat, pr):
    
    df = can_dat[can_dat['PRUID']==pr]
    
    df['centroid'] = df['geometry'].centroid
    df['distance'] = 0

    dest_df = get_dest_df(df, dest_col = 'PCUID')

    pbar = tqdm(list(range(0, df.shape[0])))
    pbar.set_description('processing {pr}'.format(pr=prr[pr]))
    
    for i in pbar: # takes about 20 mins
        temp_row = df['centroid'].iloc[i]
        df['distance'].iloc[i] = np.min(dest_df.distance(temp_row)) / 1000    

    return df


pddf = pd.DataFrame(can_dat)
qk = pddf.drop_duplicates(subset=['quadkey'], keep='first')
qk = gpd.GeoDataFrame(qk)

result_df2 = dict()
for pr in prs:
    temp_df2 = pr_dist(qk, pr)
    result_df2[pr] = temp_df2
    with open('./can-w-avg-cd-dist.p', 'wb') as file:
        pickle.dump(result_df2, file)
final_df = pd.concat(result_df2)
final_df.to_csv('./can-w-avg-cd-dist.csv', index=False)

can_merged = can_data.merge(final_df, on='quadkey', how='left')
print(can_merged.shape)
can_merged.head(5)

with open('./can-speed-tiles-dist.p', 'wb') as file:
    pickle.dump(can_merged, file)

In [93]:
qk.head(5)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,year,quarter,conn_type,PRUID,...,CDUID,CDNAME,DAUID,SACTYPE,DA_POP,PCUID,PCNAME,PCTYPE,PCCLASS,geometry
0,0023331133131332,11910,1408,27,1,1,2019,Q1,fixed,61,...,6101,Region 1,61010033,8,590.0,None,None,None,None,"POLYGON ((4593360.869 4089469.904, 4593533.055..."
1,0023331133133011,14969,1554,25,1,1,2019,Q1,fixed,61,...,6101,Region 1,61010033,8,590.0,None,None,None,None,"POLYGON ((4592705.709 4089714.238, 4592877.874..."
2,0032202103303220,5038,1317,54,1,1,2019,Q1,fixed,61,...,6101,Region 1,61010054,8,330.0,None,None,None,None,"POLYGON ((4736491.486 4146142.702, 4736658.777..."
3,0032220031120102,13419,6169,50,4,1,2019,Q1,fixed,61,...,6101,Region 1,61010045,8,275.0,None,None,None,None,"POLYGON ((4642710.684 4070147.116, 4642884.538..."
4,0032220031120103,13587,1095,28,2,1,2019,Q1,fixed,61,...,6101,Region 1,61010045,8,275.0,0388,Inuvik,4,2,"POLYGON ((4642884.538 4070012.633, 4643058.404..."


In [94]:
pddf = pd.DataFrame(can_dat)
qk = pddf.drop_duplicates(subset=['quadkey'], keep='first')
qk = gpd.GeoDataFrame(qk)
len(qk)

574971

In [95]:
result_df2 = dict()
for pr in prs:
    temp_df2 = pr_dist(qk, pr)
    result_df2[pr] = temp_df2
    with open('./can-w-avg-cd-dist.p', 'wb') as file:
        pickle.dump(result_df2, file)
final_df = pd.concat(result_df2)
final_df.to_csv('./can-w-avg-cd-dist.csv', index=False)

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


  0%|          | 0/631 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

  0%|          | 0/770 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

  0%|          | 0/55638 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()


KeyboardInterrupt: ignored

In [7]:
def parallel_df(prs):
    pr_df = Parallel(n_jobs=-1)(delayed(pr_job)(pr) for pr in prs)
    result_df = pd.concat(pr_df)

    return result_df

parallel_df(prs)

ValueError: ignored

In [13]:
len(prs)

13